In [1]:
# Comments for Patrick
#Run this cell
import pandas as pd
from folium import Map
import os
import csv
from folium.plugins import HeatMap
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import geopandas as gpd

from urllib import request
from geotext import GeoText

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

from shapely.geometry import Point, Polygon
import descartes
from tika import parser
path = '/home/hank/Work/Text.pdf' #This will be the location of your pdf or txt
PDFfolder = '/home/hank/Work/' #Location of collections of PDFs

In [2]:
pdflist = []
total = 0
lists = os.walk(PDFfolder)
for path, dir, filenames in lists:
    for filename in filenames:
        if filename.endswith(".pdf"):
                doc = os.path.join(path,filename)
                print(doc)
                total += 1 
                parsed = parser.from_file(doc)#Change the path you your own, pdf is included into the repo
                try:
                    doctext = parsed["content"]
                    docmeta = parsed["metadata"]
                    pdflist.append(doctext)

                except Exception as E:
                    print(E, "Document has no text")
                    doctext = ''
        else:
            continue
type(pdflist)

/home/hank/Work/Exit_Process Greenburg.pdf
/home/hank/Work/Text.pdf
/home/hank/Work/Exit_Process.pdf
/home/hank/Work/Testing1.pdf
/home/hank/Work/plotting-named-entities-in-python/Testing.pdf


list

In [3]:
#Run this cell to turn the list into strings so it will be able to be plotted
pdflist = str(pdflist)
type(pdflist)

str

In [4]:
#Comment this out if you want to run a pdf
#doc = open("/home/hank/Work/plotting-named-entities-in-python/text.txt",encoding="utf8").read()
#Change the path on the above code and it'll run a txt file
#print(f'{type(raw)}, \n{len(raw)}, \n{raw[:501]}')
places = GeoText(pdflist)# This is for getting a list of pdf's that have been parsed.

cities = list(places.cities)

#cities

geolocator = Nominatim(timeout=2)

lat_lon = []
for city in cities: 
    try:
        location = geolocator.geocode(city)
        if location:
            #print(location.latitude, location.longitude)
            lat_lon.append(location)
            
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"%
             (city, e)) #This is giving an error if a location is found in the document but not 
                        #able to find it's location in real life. 

/home/hank/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  # This is added back by InteractiveShellApp.init_path()


In [5]:
#This breaks up lat_lon into seperate columns or else latitude and longitude would be in the
#same column in the csv
lat = [i[0] for i in lat_lon]
lon = [i[1] for i in lat_lon]
lon1 = [i[0] for i in lon]
lon2 = [i[1] for i in lon]
#New list with the proper columns
data = {'Location':lat,'latitude': lon1, 'longitude': lon2}
#Takes our list and makes it into a dataframe
df_lat_lon = pd.DataFrame(data)
#Used to name our columns
col_names = ["Location","latitude","longitude"]
#Saves our csv to a particular location
path2 = '/home/hank/Work/Loca1.csv' #This is the location for the saved city/country names
pd.DataFrame(df_lat_lon).to_csv(path2, header=col_names,index=False)


###### This reads the csv that we saved in the cell above and uses it to plot the heatmap of our locations

In [6]:
df = pd.read_csv(path2)

In [7]:
hm_wide = HeatMap(
    list(zip(df.latitude.values, df.longitude.values)),
    min_opacity=1,
    radius=17, 
    blur=20, #If you adjust this you change the intensity of the heat map
    max_zoom=1,
)

In [8]:
for_map = Map(location=[30.169621, -96.683617], zoom_start=4, )
for_map.add_child(hm_wide)